In [2]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())



functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]



In [3]:
import openai as client

assistant = client.beta.assistants.create(
    name = "Investor Assistant",
    description= "You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
    model="gpt-4-1106-preview",
    tools=functions,
)
assistant

In [12]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain.utilities.wikipedia import WikipediaAPIWrapper
import json

#function 정의
def get_from_duckduckgo(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    keyword = inputs["keyword"]
    return ddg.run(f"Search Keyworld : {keyword}")


def get_from_wiki(inputs):
    wiki_wiki = WikipediaAPIWrapper()
    keyword = inputs["keyword"]
    return wiki_wiki.run(f"Search Keyworld : {keyword}")

functions_map = {
    "get_from_duckduckgo": get_from_duckduckgo,
    "get_from_wiki": get_from_wiki,
}

#function 스키마 정의
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_from_duckduckgo",
            "description": "Receives a keyword from the user and searches for relevant information using DuckDuckGo, then returns the result",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "The search keyword entered by the user."
                    }
                },
                "required": ["keyword"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_from_wiki",
            "description": "Receives a keyword from the user and searches for relevant information using Wikipedia API, then returns the result",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "The search keyword entered by the user."
                    }
                },
                "required": ["keyword"],
            },
        },
    },
]


In [2]:
#Asistant 생성 *생성한 후엔 아이디만 전달해주면됨.
import openai as client

assistant = client.beta.assistants.create(
    name="Search Assistant",
    instructions="You help users searching on duckduckgo or wikipedia",
    model="gpt-4-1106-preview",
    tools=functions,
)

In [3]:
assistant_id = "asst_Yya9jn7w2pFJr2DnDuPH7RPP"

In [18]:
#Thread 생성, 카톡방
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "firefox",
        }
    ]
)
thread

Thread(id='thread_H4Ul7QyglYceI9syzSmgExUt', created_at=1726409750, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [5]:
#Run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_gKREUFjCh3moEYbGDsZDWvh5', assistant_id='asst_Yya9jn7w2pFJr2DnDuPH7RPP', cancelled_at=None, completed_at=None, created_at=1726406408, expires_at=1726407008, failed_at=None, incomplete_details=None, instructions='You help users searching on duckduckgo or wikipedia', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_cSMPQVWvROZ8LYxDN6M1xelI', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_from_duckduckgo', description='Receives a keyword from the user and searches for relevant information using DuckDuckGo, then returns the result', parameters={'type': 'object', 'properties': {'keyword': {'type': 'string', 'description': 'The search keyword entered by the user.'}}, 'required': ['keyword']}, strict=False), type='function'), FunctionTool(function=

In [13]:
#그외 함수들
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )

In [14]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_from_duckduckgo with arg {"keyword": "firefox"}
Calling function: get_from_wiki with arg {"keyword": "firefox"}


Run(id='run_gKREUFjCh3moEYbGDsZDWvh5', assistant_id='asst_Yya9jn7w2pFJr2DnDuPH7RPP', cancelled_at=None, completed_at=None, created_at=1726406408, expires_at=1726407008, failed_at=None, incomplete_details=None, instructions='You help users searching on duckduckgo or wikipedia', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1726406409, status='queued', thread_id='thread_cSMPQVWvROZ8LYxDN6M1xelI', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_from_duckduckgo', description='Receives a keyword from the user and searches for relevant information using DuckDuckGo, then returns the result', parameters={'type': 'object', 'properties': {'keyword': {'type': 'string', 'description': 'The search keyword entered by the user.'}}, 'required': ['keyword']}, strict=False), type='function'), FunctionTool(fun

In [15]:
get_run(run.id, thread.id).status

'requires_action'

In [16]:
get_messages(thread.id)

user: firefox
assistant: On DuckDuckGo, I found information mostly related to using the Firefox web browser's address bar for searching, managing search engines, and about the `about:config` preferences within Firefox. Additionally, there is mention of an extension called "Keywords Everywhere," which is a browser add-on assisting users with online marketing and includes various search-related features.

The Wikipedia search did not yield a good result for "Firefox," which is unusual given that Mozilla Firefox is a well-known web browser and usually has a comprehensive Wikipedia page.

Since the Wikipedia search did not provide a satisfactory result, I will attempt to gather more specific information about Mozilla Firefox using Wikipedia's API. Please wait for a moment while I do this.
